In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
from numpy import int16
from enum import Enum

In [2]:
# Using enum class create enumerations
class Constante(Enum):
   march_1_2021 = "2021-03-01"
   specific = "2021-06-14"

In [3]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = f'''
    SELECT 
    agent_username, COUNT(*) AS nbre,
    dsd.organisation,
    dsd.created_ad as entry_date
FROM
    caris_db.dreams_surveys_data dsd
WHERE
    dsd.created_ad >= '{Constante.specific.value}'
GROUP BY agent_username
'''

agent = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

## Agent DREAMS sortant du Systeme

In [4]:
agent.rename(columns={'agent_username':'username'},inplace=True)
agent

,username,nbre,organisation,entry_date
0,1niriose,25,CARIS,2021-06-17 11:18:01
1,1yvrose,22,CARIS,2021-06-25 05:45:47
2,2eliona,12,CARIS,2021-06-18 09:54:05
3,2ernst,27,CARIS,2021-06-24 08:11:54
4,6edwina,9,CARIS,2021-06-17 09:00:46
5,6francenel,30,CARIS,2021-06-23 15:40:37
6,6georgeline,3,CARIS,2021-06-16 04:52:25
7,6lovemyschinider,28,CARIS,2021-06-25 19:21:23
8,6ludelene,9,CARIS,2021-06-20 07:04:18
9,6marckensia,6,CARIS,2021-06-15 16:30:04


In [5]:
agent_name = pd.read_excel("caris-test_users_7453177b42914ad2978e3c465e39dab1.xlsx")

In [6]:
agent_name = agent_name[["username","name"]]

## Liste de tous les agents avec Leur nom

In [7]:
agent_name

,username,name
0,1aldonzar,elcie aldonzar
1,1alexandra,Alexandra Joseph
2,1annuelle,Annuelle Eustache
3,1claudius,Claudius Gladys
4,1coldina,Coldina Duverny
...,...,...
742,s2julnor,Julnor Beaubrun
743,s2mirlene,Vernet Mirlene
744,stfleur,Jesula St Fleur
745,sylvietest,sylvie rousseau


## MERGE (Vlookup)

In [8]:
performanceAgents = agent_name.merge(agent,on="username",how="inner")

In [9]:
performanceAgents.to_excel("performancedesAgents.xlsx",index=False,na_rep="NULL")

In [10]:
performanceAgents

,username,name,nbre,organisation,entry_date
0,1niriose,Niriose Damas,25,CARIS,2021-06-17 11:18:01
1,1yvrose,Yvrose Bertrand,22,CARIS,2021-06-25 05:45:47
2,2eliona,Eliona Fredly,12,CARIS,2021-06-18 09:54:05
3,2ernst,Ernst Belabre,27,CARIS,2021-06-24 08:11:54
4,6edwina,Edwina Guerrier,9,CARIS,2021-06-17 09:00:46
5,6francenel,Francenel Dessalines,30,CARIS,2021-06-23 15:40:37
6,6georgeline,Georgeline Elouicé,3,CARIS,2021-06-16 04:52:25
7,6lovemyschinider,Lovemy Schinider Brutus,28,CARIS,2021-06-25 19:21:23
8,6ludelene,Ludelene Stalien,9,CARIS,2021-06-20 07:04:18
9,6marckensia,Marckensia Maurice,6,CARIS,2021-06-15 16:30:04
